In [12]:
from reverberation_calc import room, airDamp, material, surface, reverberationTime, DIN_18041_limits
import numpy as np
import csv

# Definition der berechneten Frequenzen
frequencies = (63, 125, 250, 500, 1000, 2000, 4000, 8000)  # Frequencies in Hz

# Definition eines Raumes
room = room(1000) # Initialisierung eines Raum-Objekts mit einem Volumen von 1000 m³
# optional Einstellung weiterer Raumparameter
room.set_temperature(20)
room.set_rel_humidity(50)
room.set_pressure(101.325)
room.set_height(5)

# Definition von Materialien (sind mehr als gebraucht werden aber hier nur als Beispiel für die Syntax bzw. Datenbankeinbindung)
def load_materials_from_csv(filepath):
    materials = {}
    frequencies = ["63", "125", "250", "500", "1000", "2000", "4000", "8000"]

    with open(filepath, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            name = row.get("name", "").strip()
            if not name:
                continue

            coeffs = []
            for freq in frequencies:
                value = row.get(freq, "").strip()
                try:
                    coeffs.append(float(value) if value else float("nan"))
                except ValueError:
                    coeffs.append(float("nan"))

            materials[name] = material(name, tuple(coeffs))

    return materials

material_dict = load_materials_from_csv("materials.csv")

material2 = material_dict.get("Teppich_6mm")

#print(material2.absorption_coefficient)

material1 = material("Kalkzementputz", (0,.03,0.03,0.02,0.04,0.05,0.05,0))
#material2 = material("Teppich", (0,0.02,0.04,0.07,0.19,0.29,0.35,0))
material3 = material("Isolierverglasung", (0,0.28,0.20,0.11,0.06,0.03,0.02,0))
material4 = material("HFPlatte_3_5", (0,0.61,0.26,0.19,0.07,0.07,0.03,0))
material5 = material("HFPlatte_4", (0,0.63,0.25,0.14,0.08,0.06,0.05,0))
material6 = material("GlaWo_Mikroperf", (0,0.75,0.95,0.95,1.00,1.00,1.00,0))
material7 = material("MetKassette_20", (0,0.38,0.65,0.59,0.75,0.77,0.61,0))
material8 = material("GK_Schlitz_8_8_Vlies", (0,0.11,0.28,0.66,0.38,0.28,0.30,0))

# Definition von zwei Beispieloberflächen
surface1 = surface("Wand1", 2000, material1)
surface2 = surface("Boden", 1, material2)

# Berechnung der Nachhallzeit aus dem Raum, den Flächen und den Frequenzen. Mit air_damp_calc wird eingestellt, ob die Luftdämpfung berücksichtigt wird.
reverberationTime_f = reverberationTime(room, [surface1,surface2], frequencies, air_damp_calc=False)
reverberationTime_t = reverberationTime(room, [surface1,surface2], frequencies, air_damp_calc=True)

# Ausgabe der Ergebnisse
print("Nachhallzeit (ohne Luftdämpfung):")
print(reverberationTime_f.reverberation_time)
print("Nachhallzeit (mit Luftdämpfung):")
print(reverberationTime_t.reverberation_time)


# Berechnung und Ausgabe der Anforderungsbereiche
print("Anforderungsbereiche nach DIN 18041:")
limits = DIN_18041_limits(room, "B1")

print("obere Grenznachhallzeiten: " + str(limits.T_upper_limit))
print("untere Grenznachhallzeiten: " + str(limits.T_lower_limit))


[ nan 0.02 0.04 0.07 0.19 0.29 0.35  nan]
Nachhallzeit (ohne Luftdämpfung):
[       nan 2.68617944 2.68528464 4.023571   2.01053111 1.60758291
 1.60662172        nan]
Nachhallzeit (mit Luftdämpfung):
[       nan 2.66820155 2.63247864 3.786512   1.90844719 1.47396477
 1.26323121        nan]
Anforderungsbereiche nach DIN 18041:
obere Grenznachhallzeiten: [0 0 0 0 0 0 0 0]
untere Grenznachhallzeiten: [0 0 0 0 0 0 0 0]


/Users/paula/Documents/TU/Semester04/Python&Akustik/Raumakustik-CAD/reverberation_calc.py:461: UserWarning: According to DIN 18041, there are no requirements for type B1.
  warnings.warn("According to DIN 18041, there are no requirements for type B1.")
